In [1]:
import pandas as pd
#import xlwings as xw
#import xlsxwriter
import os
import pickle
import math
import numpy as np
import scipy.stats as st

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')   
os.chdir("/content/gdrive/My Drive/")


Mounted at /content/gdrive


In [3]:
#files = os.listdir("HP comp")
BASE_DIR =  'MIE490: Capstone/Data/chronological/'
#files = os.listdir(BASE_DIR+'Male/')

# files_to_process = []
# for file in files:
#     if 'xlsx' in file and '.DS_Store' not in file and '~$' not in file:
#         files_to_process.append(file)

# files_to_process
os.listdir(BASE_DIR+'Male/')

['M_1996.xlsx',
 'M_2011.xlsx',
 'M_2007.xlsx',
 'M_2010.xlsx',
 'M_2006.xlsx',
 'M_1997.xlsx',
 'M_2001.xlsx',
 'M_2017.xlsx',
 'M_2000.xlsx',
 'M_2016.xlsx',
 'M_2003.xlsx',
 'M_2015.xlsx',
 'M_2019.xlsx',
 'M_2018.xlsx',
 'M_2014.xlsx',
 'M_2009.xlsx',
 'M_1998.xlsx',
 'M_2005.xlsx',
 'M_2013.xlsx',
 'M_1995.xlsx',
 'M_2012.xlsx',
 'M_2004.xlsx',
 'M_1999.xlsx',
 'M_2008.xlsx',
 'M_2002.xlsx']

## KNN classification

In [5]:
# load the aggragated athletes data
# two sheets: Champions, Records
# Champions contains the athletes who won gold or podium
agg_record = pd.ExcelFile('MIE490: Capstone/Data/' + 'ChampionList.xlsx')
track_records = agg_record.parse('Records')

In [6]:
# load the track of Elo records for both Male and Female
records_male = track_records[track_records['Gender'] == 'M'].copy()
records_female = track_records[track_records['Gender'] == 'L'].copy()

In [7]:
# organize by the number of years of records
# will be used to calculate KNN
records_dict_M ={}
for i in range (2,20):
    temp = records_male[~records_male['Year{} end score'.format(i)].isnull()].copy()
    temp = temp.reset_index(drop=True)
    records_dict_M[i] = temp

In [8]:
# organize by the number of years of records
# will be used to calculate KNN
records_dict_F ={}
for i in range (2,20):
    temp = records_female[~records_female['Year{} end score'.format(i)].isnull()].copy()
    temp = temp.reset_index(drop=True)
    records_dict_F[i] = temp

In [9]:
# Euclidean distance
def distance(row, row_test):
    row_train = row.values
    return np.linalg.norm(row_test-row_train)

In [10]:
def KNN_classification(test_data, data, k, year):
    count = 0
    # use the index to filter the records for all athletes
    start_index = data.columns.get_loc('Year1 end score')
    end_index = start_index + year
    for i in test_data.index:
        # iterate through all athlete, classify one at a time
        data_test = test_data.loc[i, :].copy()
        # Use the rest of athletes as training data
        data_train = data[~data.index.isin([i])].copy()

        # records of athelte who we want to classify as winner or not
        row_test = data_test.iloc[start_index:end_index].values
        # calculate distance between athletes' records
        # calculate distance only based on the same number of records as the test athlete's
        data_train['dist'] = data_train.iloc[:,start_index:end_index].apply(distance, axis=1, row_test=row_test)
        
        # get top K nearest neighbours
        sorted_data_train = data_train.sort_values(by='dist', ascending=True).iloc[:k,:]

        # get KNN classification
        # the majority of the athlete type (winner or non-winner) is the prediction for the current athlete
        classification = sorted_data_train['Podium (Y/N)'].value_counts().keys()[0]

        count += data_test['Podium (Y/N)'] == classification

    return count/test_data.shape[0]


In [11]:
'''
The following KNN algorithm predict on every athlete's potential to become a podium winner based on the competition history to others in the dataset.

Everytime, the predicting athlete is separated from the rest of athletes.
Because the KNN algorithm is run on the fly, so this model does not need training process.
Thus, there is no train/test split for this function (but we performed train/test split later, and proven not affecting the final results).


'''
def run_KNN(records, k, year):
    # filter by the year of getting first podium -> for path prediction
    KNN_data = records[year]
    # filter the winners by those who have at least KNN_data_year to get a podium
    # this can be used to find similar athlete's trajectory towards a podium
    KNN_data.loc[KNN_data['Year last podium']<year, 'Podium (Y/N)'] = 'No'
    winners = KNN_data[KNN_data['Podium (Y/N)']=='Yes'].copy()
    non_winners = KNN_data[KNN_data['Podium (Y/N)']=='No'].copy()

    TP_result = []
    TN_result = []
    Acc_result = []
    for i in range(10):
        # create 10 cross-validation balanced dataset for KNN classification
        sample_non_winners = non_winners.sample(n=winners.shape[0])
        balanced_data = pd.concat([sample_non_winners, winners])
        TP = KNN_classification(winners,balanced_data, k, year)
        TN = KNN_classification(sample_non_winners,balanced_data, k, year)
        TP_result.append(TP)
        TN_result.append(TN)
        Acc_result.append((TP+TN)/2)
    print("There are {} winners out of {} athletes".format(sample_non_winners.shape[0], KNN_data.shape[0]))
    return sum(TP_result)/len(TP_result), sum(TN_result)/len(TN_result),sum(Acc_result)/len(Acc_result)
        

## KNN results - Male

In [ ]:
KNN_data_year = 3
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_M, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 106 winners out of 1097 athletes
Precision(True positive): 0.7707547169811322, True negative:0.8009433962264151, Accuracy:0.7858490566037737,  year: 3, K: 3
There are 106 winners out of 1097 athletes
Precision(True positive): 0.6924528301886792, True negative:0.868867924528302, Accuracy:0.7806603773584906,  year: 3, K: 4
There are 106 winners out of 1097 athletes
Precision(True positive): 0.7877358490566039, True negative:0.7971698113207548, Accuracy:0.7924528301886793,  year: 3, K: 5
There are 106 winners out of 1097 athletes
Precision(True positive): 0.7283018867924528, True negative:0.8528301886792453, Accuracy:0.790566037735849,  year: 3, K: 6
There are 106 winners out of 1097 athletes
Precision(True positive): 0.7801886792452831, True negative:0.8226415094339622, Accuracy:0.8014150943396225,  year: 3, K: 7


In [ ]:
KNN_data_year = 4
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_M, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 86 winners out of 784 athletes
Precision(True positive): 0.7593023255813952, True negative:0.7569767441860465, Accuracy:0.758139534883721,  year: 4, K: 3
There are 86 winners out of 784 athletes
Precision(True positive): 0.6465116279069768, True negative:0.8220930232558139, Accuracy:0.7343023255813954,  year: 4, K: 4
There are 86 winners out of 784 athletes
Precision(True positive): 0.7488372093023258, True negative:0.7651162790697674, Accuracy:0.7569767441860467,  year: 4, K: 5
There are 86 winners out of 784 athletes
Precision(True positive): 0.7058139534883721, True negative:0.8081395348837208, Accuracy:0.7569767441860467,  year: 4, K: 6
There are 86 winners out of 784 athletes
Precision(True positive): 0.7523255813953489, True negative:0.7558139534883722, Accuracy:0.7540697674418604,  year: 4, K: 7


In [ ]:
KNN_data_year = 5
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_M, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 71 winners out of 549 athletes
Precision(True positive): 0.7943661971830986, True negative:0.776056338028169, Accuracy:0.7852112676056338,  year: 5, K: 3
There are 71 winners out of 549 athletes
Precision(True positive): 0.7183098591549296, True negative:0.8366197183098592, Accuracy:0.7774647887323944,  year: 5, K: 4
There are 71 winners out of 549 athletes
Precision(True positive): 0.823943661971831, True negative:0.804225352112676, Accuracy:0.8140845070422534,  year: 5, K: 5
There are 71 winners out of 549 athletes
Precision(True positive): 0.7690140845070422, True negative:0.819718309859155, Accuracy:0.7943661971830986,  year: 5, K: 6
There are 71 winners out of 549 athletes
Precision(True positive): 0.808450704225352, True negative:0.8070422535211268, Accuracy:0.8077464788732394,  year: 5, K: 7


In [ ]:
KNN_data_year = 6
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_M, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 59 winners out of 406 athletes
Precision(True positive): 0.7864406779661016, True negative:0.7440677966101694, Accuracy:0.7652542372881357,  year: 6, K: 3
There are 59 winners out of 406 athletes
Precision(True positive): 0.6728813559322033, True negative:0.7915254237288135, Accuracy:0.7322033898305085,  year: 6, K: 4
There are 59 winners out of 406 athletes
Precision(True positive): 0.7694915254237287, True negative:0.7542372881355931, Accuracy:0.7618644067796609,  year: 6, K: 5
There are 59 winners out of 406 athletes
Precision(True positive): 0.7389830508474575, True negative:0.7966101694915253, Accuracy:0.7677966101694916,  year: 6, K: 6
There are 59 winners out of 406 athletes
Precision(True positive): 0.8152542372881356, True negative:0.7694915254237287, Accuracy:0.7923728813559322,  year: 6, K: 7


In [ ]:
KNN_data_year = 7
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_M, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 45 winners out of 283 athletes
Precision(True positive): 0.7088888888888889, True negative:0.6977777777777778, Accuracy:0.7033333333333334,  year: 7, K: 3
There are 45 winners out of 283 athletes
Precision(True positive): 0.8244444444444443, True negative:0.6577777777777777, Accuracy:0.7411111111111112,  year: 7, K: 4
There are 45 winners out of 283 athletes
Precision(True positive): 0.7577777777777778, True negative:0.6799999999999999, Accuracy:0.718888888888889,  year: 7, K: 5
There are 45 winners out of 283 athletes
Precision(True positive): 0.8177777777777777, True negative:0.6511111111111112, Accuracy:0.7344444444444445,  year: 7, K: 6
There are 45 winners out of 283 athletes
Precision(True positive): 0.791111111111111, True negative:0.7000000000000001, Accuracy:0.7455555555555555,  year: 7, K: 7


## KNN results - Female

In [ ]:
KNN_data_year = 3
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_F, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 70 winners out of 563 athletes
Precision(True positive): 0.8242857142857142, True negative:0.7528571428571429, Accuracy:0.7885714285714285,  year: 3, K: 3
There are 70 winners out of 563 athletes
Precision(True positive): 0.8785714285714287, True negative:0.6871428571428573, Accuracy:0.7828571428571427,  year: 3, K: 4
There are 70 winners out of 563 athletes
Precision(True positive): 0.7885714285714285, True negative:0.7828571428571427, Accuracy:0.7857142857142856,  year: 3, K: 5
There are 70 winners out of 563 athletes
Precision(True positive): 0.8285714285714285, True negative:0.7614285714285713, Accuracy:0.7949999999999999,  year: 3, K: 6
There are 70 winners out of 563 athletes
Precision(True positive): 0.8099999999999999, True negative:0.8128571428571428, Accuracy:0.8114285714285714,  year: 3, K: 7


In [ ]:
KNN_data_year = 4
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_F, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 61 winners out of 393 athletes
Precision(True positive): 0.8327868852459016, True negative:0.7918032786885247, Accuracy:0.812295081967213,  year: 4, K: 3
There are 61 winners out of 393 athletes
Precision(True positive): 0.9049180327868852, True negative:0.7278688524590164, Accuracy:0.8163934426229508,  year: 4, K: 4
There are 61 winners out of 393 athletes
Precision(True positive): 0.821311475409836, True negative:0.8098360655737704, Accuracy:0.8155737704918034,  year: 4, K: 5
There are 61 winners out of 393 athletes
Precision(True positive): 0.8590163934426229, True negative:0.7557377049180328, Accuracy:0.8073770491803278,  year: 4, K: 6
There are 61 winners out of 393 athletes
Precision(True positive): 0.7786885245901639, True negative:0.8, Accuracy:0.789344262295082,  year: 4, K: 7


In [ ]:
KNN_data_year = 5
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_F, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 52 winners out of 285 athletes
Precision(True positive): 0.7557692307692307, True negative:0.7692307692307692, Accuracy:0.7625,  year: 5, K: 3
There are 52 winners out of 285 athletes
Precision(True positive): 0.846153846153846, True negative:0.7442307692307693, Accuracy:0.7951923076923076,  year: 5, K: 4
There are 52 winners out of 285 athletes
Precision(True positive): 0.7903846153846154, True negative:0.8269230769230769, Accuracy:0.8086538461538462,  year: 5, K: 5
There are 52 winners out of 285 athletes
Precision(True positive): 0.8346153846153845, True negative:0.798076923076923, Accuracy:0.8163461538461538,  year: 5, K: 6
There are 52 winners out of 285 athletes
Precision(True positive): 0.7961538461538462, True negative:0.7961538461538461, Accuracy:0.7961538461538462,  year: 5, K: 7


In [ ]:
KNN_data_year = 6
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_F, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 39 winners out of 211 athletes
Precision(True positive): 0.7025641025641026, True negative:0.7589743589743589, Accuracy:0.7307692307692308,  year: 6, K: 3
There are 39 winners out of 211 athletes
Precision(True positive): 0.8205128205128205, True negative:0.7076923076923076, Accuracy:0.764102564102564,  year: 6, K: 4
There are 39 winners out of 211 athletes
Precision(True positive): 0.6692307692307693, True negative:0.7769230769230769, Accuracy:0.7230769230769232,  year: 6, K: 5
There are 39 winners out of 211 athletes
Precision(True positive): 0.8051282051282052, True negative:0.764102564102564, Accuracy:0.7846153846153845,  year: 6, K: 6
There are 39 winners out of 211 athletes
Precision(True positive): 0.7666666666666667, True negative:0.7615384615384615, Accuracy:0.764102564102564,  year: 6, K: 7


In [ ]:
KNN_data_year = 7
for k in [3,4,5,6,7]:
    KNN_results = run_KNN(records_dict_F, k,KNN_data_year)
    print('Precision(True positive): {}, True negative:{}, Accuracy:{},  year: {}, K: {}'.format(KNN_results[0], KNN_results[1],KNN_results[2], KNN_data_year, k))

There are 29 winners out of 155 athletes
Precision(True positive): 0.7172413793103448, True negative:0.7793103448275863, Accuracy:0.7482758620689656,  year: 7, K: 3
There are 29 winners out of 155 athletes
Precision(True positive): 0.8413793103448276, True negative:0.6965517241379311, Accuracy:0.7689655172413793,  year: 7, K: 4
There are 29 winners out of 155 athletes
Precision(True positive): 0.7793103448275863, True negative:0.7793103448275862, Accuracy:0.7793103448275861,  year: 7, K: 5
There are 29 winners out of 155 athletes
Precision(True positive): 0.8241379310344825, True negative:0.7137931034482758, Accuracy:0.7689655172413794,  year: 7, K: 6
There are 29 winners out of 155 athletes
Precision(True positive): 0.7724137931034483, True negative:0.7482758620689656, Accuracy:0.7603448275862069,  year: 7, K: 7


## Train/Test split

In [12]:
def run_KNN_train_test(records, k, year):
    # filter by the year of getting first podium -> for path prediction
    KNN_data = records[year]
    # filter the winners by those who have at least KNN_data_year to get a podium
    # this can be used to find similar athlete's trajectory towards a podium
    KNN_data.loc[KNN_data['Year last podium']<year, 'Podium (Y/N)'] = 'No'
    winners = KNN_data[KNN_data['Podium (Y/N)']=='Yes'].copy()
    non_winners = KNN_data[KNN_data['Podium (Y/N)']=='No'].copy()

    TP_result_train = []
    TN_result_train = []
    Acc_result_train = []
    TP_result_test = []
    TN_result_test = []
    Acc_result_test = []
    for i in range(10):
        # create 10 cross-validation balanced dataset for KNN classification

        winner_train_index = list(winners.sample(n=int(winners.shape[0]*0.7)).index)
        winner_train = winners.loc[winners.index.isin(winner_train_index)]
        winner_test = winners.loc[~winners.index.isin(winner_train_index)]

        non_winner_train = non_winners.sample(n=winner_train.shape[0])
        non_winner_test = non_winners.loc[~non_winners.index.isin(non_winner_train)].sample(n=winner_test.shape[0])
        balanced_data_train = pd.concat([non_winner_train, winner_train])
        
        
        TP_train = KNN_classification(winner_train, balanced_data_train, k, year)
        TN_train = KNN_classification(non_winner_train, balanced_data_train, k, year)
        TP_result_train.append(TP_train)
        TN_result_train.append(TN_train)
        Acc_result_train.append((TP_train+TN_train)/2)

        # test KNN classification for the "testing athletes" on the training data
        TP_test = KNN_classification(winner_test, balanced_data_train, k, year)
        TN_test = KNN_classification(non_winner_test, balanced_data_train, k, year)
        TP_result_test.append(TP_test)
        TN_result_test.append(TN_test)
        Acc_result_test.append((TP_test+TN_test)/2)

    # compute 95% confidence interval using t-distribution
    CI_train = st.t.interval(alpha=0.95, df=len(Acc_result_train)-1, loc=np.mean(Acc_result_train), scale=st.sem(Acc_result_train))
    CI_test = st.t.interval(alpha=0.95, df=len(Acc_result_test)-1, loc=np.mean(Acc_result_test), scale=st.sem(Acc_result_test))


    print("There are {} winners(training) + {} winners(testing) out of {} athletes".format(winner_train.shape[0],winner_test.shape[0], KNN_data.shape[0]))
    return sum(TP_result_train)/len(TP_result_train), sum(TN_result_train)/len(TN_result_train),sum(Acc_result_train)/len(Acc_result_train), CI_train[0], CI_train[1],\
           sum(TP_result_test)/len(TP_result_test),   sum(TN_result_test)/len(TN_result_test),  sum(Acc_result_test)/len(Acc_result_test),   CI_test[0],  CI_test[1]
        

In [20]:
KNN_data_year = 3
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_M, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)','TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 74 winners(training) + 32 winners(testing) out of 1097 athletes
year: 3, K: 3
 Training: Precision(True positive): 0.754054054054054, True negative:0.7918918918918919, Accuracy:0.772972972972973
     Test: Precision(True positive): 0.7544340800282747, True negative:0.791511865917671, Accuracy:0.796875

There are 74 winners(training) + 32 winners(testing) out of 1097 athletes
year: 3, K: 4
 Training: Precision(True positive): 0.6851351351351351, True negative:0.8567567567567569, Accuracy:0.7709459459459459
     Test: Precision(True positive): 0.7388477699881345, True negative:0.8030441219037573, Accuracy:0.69375

There are 74 winners(training) + 32 winners(testing) out of 1097 athletes
year: 3, K: 5
 Training: Precision(True positive): 0.758108108108108, True negative:0.7837837837837838, Accuracy:0.770945945945946
     Test: Precision(True positive): 0.747996044360107, True negative:0.793895847531785, Accuracy:0.76875

There are 74 winners(training) + 32 winners(testing) out o

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.754054,0.791892,0.772973,0.754434,0.791512,0.796875,0.79375,0.795312,0.755029,0.835596
1,0.685135,0.856757,0.770946,0.738848,0.803044,0.693750,0.86875,0.781250,0.757686,0.804814
2,0.758108,0.783784,0.770946,0.747996,0.793896,0.768750,0.77500,0.771875,0.742919,0.800831
3,0.720270,0.843243,0.781757,0.755180,0.808334,0.756250,0.85000,0.803125,0.766925,0.839325
4,0.774324,0.817568,0.795946,0.763578,0.828314,0.762500,0.82500,0.793750,0.773479,0.814021


In [21]:
KNN_data_year = 4
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_M, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)', 'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 60 winners(training) + 26 winners(testing) out of 784 athletes
year: 4, K: 3
 Training: Precision(True positive): 0.7516666666666667, True negative:0.7483333333333334, Accuracy:0.75
     Test: Precision(True positive): 0.7185811505174862, True negative:0.7814188494825138, Accuracy:0.8038461538461539

There are 60 winners(training) + 26 winners(testing) out of 784 athletes
year: 4, K: 4
 Training: Precision(True positive): 0.7066666666666667, True negative:0.8433333333333332, Accuracy:0.7749999999999999
     Test: Precision(True positive): 0.7478995348289589, True negative:0.8021004651710412, Accuracy:0.6653846153846155

There are 60 winners(training) + 26 winners(testing) out of 784 athletes
year: 4, K: 5
 Training: Precision(True positive): 0.7516666666666667, True negative:0.7616666666666666, Accuracy:0.7566666666666668
     Test: Precision(True positive): 0.721142519653322, True negative:0.7921908136800114, Accuracy:0.7499999999999999

There are 60 winners(training) + 26 w

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.751667,0.748333,0.750000,0.718581,0.781419,0.803846,0.750000,0.776923,0.743101,0.810745
1,0.706667,0.843333,0.775000,0.747900,0.802100,0.665385,0.811538,0.738462,0.696334,0.780589
2,0.751667,0.761667,0.756667,0.721143,0.792191,0.750000,0.792308,0.771154,0.729909,0.812399
3,0.708333,0.823333,0.765833,0.735120,0.796547,0.715385,0.838462,0.776923,0.732844,0.821002
4,0.771667,0.765000,0.768333,0.748502,0.788165,0.750000,0.773077,0.761538,0.723063,0.800014


In [22]:
KNN_data_year = 5
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_M, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)' ,'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 49 winners(training) + 22 winners(testing) out of 549 athletes
year: 5, K: 3
 Training: Precision(True positive): 0.8061224489795917, True negative:0.7551020408163266, Accuracy:0.7806122448979591
     Test: Precision(True positive): 0.7486549512420146, True negative:0.8125695385539036, Accuracy:0.8454545454545455

There are 49 winners(training) + 22 winners(testing) out of 549 athletes
year: 5, K: 4
 Training: Precision(True positive): 0.7061224489795919, True negative:0.8367346938775512, Accuracy:0.7714285714285714
     Test: Precision(True positive): 0.7432203906608241, True negative:0.7996367521963188, Accuracy:0.6500000000000001

There are 49 winners(training) + 22 winners(testing) out of 549 athletes
year: 5, K: 5
 Training: Precision(True positive): 0.8081632653061226, True negative:0.7938775510204081, Accuracy:0.8010204081632653
     Test: Precision(True positive): 0.7721791895913871, True negative:0.8298616267351439, Accuracy:0.8045454545454545

There are 49 winners(t

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.806122,0.755102,0.780612,0.748655,0.812570,0.845455,0.768182,0.806818,0.752489,0.861147
1,0.706122,0.836735,0.771429,0.743220,0.799637,0.650000,0.854545,0.752273,0.704749,0.799797
2,0.808163,0.793878,0.801020,0.772179,0.829862,0.804545,0.795455,0.800000,0.765897,0.834103
3,0.702041,0.818367,0.760204,0.722944,0.797464,0.677273,0.863636,0.770455,0.716018,0.824892
4,0.795918,0.767347,0.781633,0.756299,0.806966,0.836364,0.777273,0.806818,0.743968,0.869669


In [23]:
KNN_data_year = 6
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_M, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)', 'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 41 winners(training) + 18 winners(testing) out of 406 athletes
year: 6, K: 3
 Training: Precision(True positive): 0.7658536585365853, True negative:0.7487804878048779, Accuracy:0.7573170731707316
     Test: Precision(True positive): 0.7108898632958036, True negative:0.8037442830456596, Accuracy:0.7500000000000001

There are 41 winners(training) + 18 winners(testing) out of 406 athletes
year: 6, K: 4
 Training: Precision(True positive): 0.6804878048780487, True negative:0.8292682926829269, Accuracy:0.7548780487804878
     Test: Precision(True positive): 0.7162011553799343, True negative:0.7935549421810413, Accuracy:0.6944444444444444

There are 41 winners(training) + 18 winners(testing) out of 406 athletes
year: 6, K: 5
 Training: Precision(True positive): 0.7658536585365853, True negative:0.7487804878048779, Accuracy:0.7573170731707316
     Test: Precision(True positive): 0.719749266926784, True negative:0.7948848794146794, Accuracy:0.7777777777777778

There are 41 winners(tr

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.765854,0.748780,0.757317,0.710890,0.803744,0.750000,0.750000,0.750000,0.708108,0.791892
1,0.680488,0.829268,0.754878,0.716201,0.793555,0.694444,0.777778,0.736111,0.698350,0.773872
2,0.765854,0.748780,0.757317,0.719749,0.794885,0.777778,0.733333,0.755556,0.713874,0.797237
3,0.668293,0.770732,0.719512,0.675603,0.763421,0.722222,0.783333,0.752778,0.701943,0.803612
4,0.768293,0.739024,0.753659,0.712370,0.794947,0.833333,0.722222,0.777778,0.723158,0.832398


In [24]:
KNN_data_year = 7
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_M, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)', 'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 31 winners(training) + 14 winners(testing) out of 283 athletes
year: 7, K: 3
 Training: Precision(True positive): 0.7064516129032259, True negative:0.7129032258064518, Accuracy:0.7096774193548387
     Test: Precision(True positive): 0.6723890186682802, True negative:0.7469658200413973, Accuracy:0.7571428571428571

There are 31 winners(training) + 14 winners(testing) out of 283 athletes
year: 7, K: 4
 Training: Precision(True positive): 0.5903225806451613, True negative:0.7612903225806452, Accuracy:0.6758064516129033
     Test: Precision(True positive): 0.6021376363210249, True negative:0.7494752669047817, Accuracy:0.5857142857142857

There are 31 winners(training) + 14 winners(testing) out of 283 athletes
year: 7, K: 5
 Training: Precision(True positive): 0.7193548387096775, True negative:0.7129032258064517, Accuracy:0.7161290322580647
     Test: Precision(True positive): 0.6490275437878951, True negative:0.7832305207282338, Accuracy:0.7642857142857142

There are 31 winners(t

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.706452,0.712903,0.709677,0.672389,0.746966,0.757143,0.685714,0.721429,0.670616,0.772241
1,0.590323,0.761290,0.675806,0.602138,0.749475,0.585714,0.778571,0.682143,0.623821,0.740465
2,0.719355,0.712903,0.716129,0.649028,0.783231,0.764286,0.678571,0.721429,0.670616,0.772241
3,0.593548,0.741935,0.667742,0.626606,0.708878,0.650000,0.664286,0.657143,0.577984,0.736302
4,0.709677,0.703226,0.706452,0.651574,0.761330,0.735714,0.714286,0.725000,0.651692,0.798308


### Female Train/test results

In [25]:
KNN_data_year = 3
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_F, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)', 'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 49 winners(training) + 21 winners(testing) out of 563 athletes
year: 3, K: 3
 Training: Precision(True positive): 0.753061224489796, True negative:0.7428571428571429, Accuracy:0.7479591836734694
     Test: Precision(True positive): 0.7154873059112423, True negative:0.7804310614356964, Accuracy:0.7904761904761904

There are 49 winners(training) + 21 winners(testing) out of 563 athletes
year: 3, K: 4
 Training: Precision(True positive): 0.6714285714285715, True negative:0.8489795918367348, Accuracy:0.7602040816326531
     Test: Precision(True positive): 0.7328374903262993, True negative:0.7875706729390068, Accuracy:0.6000000000000001

There are 49 winners(training) + 21 winners(testing) out of 563 athletes
year: 3, K: 5
 Training: Precision(True positive): 0.7469387755102039, True negative:0.810204081632653, Accuracy:0.7785714285714286
     Test: Precision(True positive): 0.7357238457580619, True negative:0.8214190113847954, Accuracy:0.7571428571428571

There are 49 winners(tra

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.753061,0.742857,0.747959,0.715487,0.780431,0.790476,0.719048,0.754762,0.708603,0.800921
1,0.671429,0.848980,0.760204,0.732837,0.787571,0.600000,0.904762,0.752381,0.697807,0.806955
2,0.746939,0.810204,0.778571,0.735724,0.821419,0.757143,0.814286,0.785714,0.752609,0.818819
3,0.693878,0.842857,0.768367,0.740402,0.796333,0.757143,0.828571,0.792857,0.781361,0.804353
4,0.791837,0.795918,0.793878,0.760551,0.827204,0.709524,0.823810,0.766667,0.730048,0.803285


In [26]:
KNN_data_year = 4
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_F, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)' ,'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 42 winners(training) + 19 winners(testing) out of 393 athletes
year: 4, K: 3
 Training: Precision(True positive): 0.7880952380952381, True negative:0.7714285714285715, Accuracy:0.7797619047619048
     Test: Precision(True positive): 0.7504666663177526, True negative:0.8090571432060569, Accuracy:0.7999999999999999

There are 42 winners(training) + 19 winners(testing) out of 393 athletes
year: 4, K: 4
 Training: Precision(True positive): 0.6738095238095239, True negative:0.869047619047619, Accuracy:0.7714285714285714
     Test: Precision(True positive): 0.7218037555646524, True negative:0.8210533872924903, Accuracy:0.631578947368421

There are 42 winners(training) + 19 winners(testing) out of 393 athletes
year: 4, K: 5
 Training: Precision(True positive): 0.7738095238095238, True negative:0.780952380952381, Accuracy:0.7773809523809523
     Test: Precision(True positive): 0.739074034891856, True negative:0.8156878698700488, Accuracy:0.8105263157894737

There are 42 winners(train

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.788095,0.771429,0.779762,0.750467,0.809057,0.800000,0.794737,0.797368,0.777426,0.817311
1,0.673810,0.869048,0.771429,0.721804,0.821053,0.631579,0.842105,0.736842,0.669258,0.804427
2,0.773810,0.780952,0.777381,0.739074,0.815688,0.810526,0.736842,0.773684,0.716723,0.830646
3,0.669048,0.814286,0.741667,0.720029,0.763304,0.715789,0.805263,0.760526,0.721389,0.799664
4,0.788095,0.807143,0.797619,0.771602,0.823636,0.763158,0.815789,0.789474,0.737728,0.841219


In [27]:
KNN_data_year = 5
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_F, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)' ,'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 36 winners(training) + 16 winners(testing) out of 285 athletes
year: 5, K: 3
 Training: Precision(True positive): 0.7694444444444444, True negative:0.7777777777777777, Accuracy:0.7736111111111111
     Test: Precision(True positive): 0.7306720168183423, True negative:0.81655020540388, Accuracy:0.6625

There are 36 winners(training) + 16 winners(testing) out of 285 athletes
year: 5, K: 4
 Training: Precision(True positive): 0.6972222222222223, True negative:0.8694444444444442, Accuracy:0.7833333333333333
     Test: Precision(True positive): 0.7575945809226172, True negative:0.8090720857440497, Accuracy:0.7375

There are 36 winners(training) + 16 winners(testing) out of 285 athletes
year: 5, K: 5
 Training: Precision(True positive): 0.8055555555555554, True negative:0.85, Accuracy:0.8277777777777778
     Test: Precision(True positive): 0.7936160492665986, True negative:0.8619395062889571, Accuracy:0.775

There are 36 winners(training) + 16 winners(testing) out of 285 athletes
ye

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.769444,0.777778,0.773611,0.730672,0.816550,0.66250,0.81250,0.737500,0.690137,0.784863
1,0.697222,0.869444,0.783333,0.757595,0.809072,0.73750,0.81875,0.778125,0.719974,0.836276
2,0.805556,0.850000,0.827778,0.793616,0.861940,0.77500,0.81250,0.793750,0.738789,0.848711
3,0.750000,0.833333,0.791667,0.741659,0.841674,0.73125,0.83750,0.784375,0.704500,0.864250
4,0.805556,0.808333,0.806944,0.778282,0.835607,0.75000,0.81875,0.784375,0.724344,0.844406


In [28]:
KNN_data_year = 6
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_F, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)', 'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 27 winners(training) + 12 winners(testing) out of 211 athletes
year: 6, K: 3
 Training: Precision(True positive): 0.7037037037037038, True negative:0.825925925925926, Accuracy:0.7648148148148148
     Test: Precision(True positive): 0.7219794400085426, True negative:0.807650189621087, Accuracy:0.7666666666666667

There are 27 winners(training) + 12 winners(testing) out of 211 athletes
year: 6, K: 4
 Training: Precision(True positive): 0.5444444444444444, True negative:0.837037037037037, Accuracy:0.6907407407407407
     Test: Precision(True positive): 0.658867470773686, True negative:0.7226140107077953, Accuracy:0.6166666666666667

There are 27 winners(training) + 12 winners(testing) out of 211 athletes
year: 6, K: 5
 Training: Precision(True positive): 0.6703703703703705, True negative:0.7481481481481482, Accuracy:0.7092592592592594
     Test: Precision(True positive): 0.6633478518141136, True negative:0.7551706667044047, Accuracy:0.775

There are 27 winners(training) + 12 win

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.703704,0.825926,0.764815,0.721979,0.807650,0.766667,0.750000,0.758333,0.695811,0.820856
1,0.544444,0.837037,0.690741,0.658867,0.722614,0.616667,0.833333,0.725000,0.661849,0.788151
2,0.670370,0.748148,0.709259,0.663348,0.755171,0.775000,0.783333,0.779167,0.739301,0.819033
3,0.588889,0.777778,0.683333,0.629450,0.737217,0.616667,0.808333,0.712500,0.630029,0.794971
4,0.703704,0.737037,0.720370,0.664358,0.776383,0.783333,0.775000,0.779167,0.726499,0.831834


In [29]:
KNN_data_year = 7
R = []
for k in [3,4,5,6,7]:
    KNN_results = run_KNN_train_test(records_dict_F, k,KNN_data_year)
    R.append(list(KNN_results))
    print('year: {}, K: {}\n Training: Precision(True positive): {}, True negative:{}, Accuracy:{}\n \
    Test: Precision(True positive): {}, True negative:{}, Accuracy:{}\n'.format(KNN_data_year, k, KNN_results[0], KNN_results[1],KNN_results[2],KNN_results[3], KNN_results[4],KNN_results[5],KNN_results[6],KNN_results[7],KNN_results[8],KNN_results[9]))

pd.DataFrame(R, columns=['TP(train)', 'TN(train)', 'Acc(train)','CI_low(train)','CI_up(train)' ,'TP(test)','TN(test)','Acc(test)','CI_low(test)', 'CI_up(test)'])

There are 20 winners(training) + 9 winners(testing) out of 155 athletes
year: 7, K: 3
 Training: Precision(True positive): 0.73, True negative:0.76, Accuracy:0.745
     Test: Precision(True positive): 0.6952668293390835, True negative:0.7947331706609165, Accuracy:0.6777777777777778

There are 20 winners(training) + 9 winners(testing) out of 155 athletes
year: 7, K: 4
 Training: Precision(True positive): 0.535, True negative:0.805, Accuracy:0.67
     Test: Precision(True positive): 0.6117131649151099, True negative:0.7282868350848902, Accuracy:0.5111111111111112

There are 20 winners(training) + 9 winners(testing) out of 155 athletes
year: 7, K: 5
 Training: Precision(True positive): 0.6599999999999999, True negative:0.71, Accuracy:0.6849999999999999
     Test: Precision(True positive): 0.6426789147099329, True negative:0.727321085290067, Accuracy:0.8666666666666666

There are 20 winners(training) + 9 winners(testing) out of 155 athletes
year: 7, K: 6
 Training: Precision(True positive)

,TP(train),TN(train),Acc(train),CI_low(train),CI_up(train),TP(test),TN(test),Acc(test),CI_low(test),CI_up(test)
0,0.730,0.760,0.7450,0.695267,0.794733,0.677778,0.766667,0.722222,0.676332,0.768112
1,0.535,0.805,0.6700,0.611713,0.728287,0.511111,0.944444,0.727778,0.664384,0.791172
2,0.660,0.710,0.6850,0.642679,0.727321,0.866667,0.666667,0.766667,0.679194,0.854139
3,0.610,0.775,0.6925,0.625558,0.759442,0.688889,0.744444,0.716667,0.653273,0.780060
4,0.810,0.655,0.7325,0.667716,0.797284,0.800000,0.688889,0.744444,0.690797,0.798092
